In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from DDBSCAN import Raster_DBSCAN
import time
from sklearn.cluster import DBSCAN

In [ ]:
Td_maps = []
pcap_file_path = r'D:\LiDAR_Data\9thVir\2024-03-14-23-30-00.pcap'
# pcap_file_path = r'D:\LiDAR_Data\9thVir\2024-03-15-01-30-00.pcap'
packets_gen = read_packets_offline(pcap_file_path)
packet = next(packets_gen)
Initial_ts,raw_packet = packet

packets_gen = read_packets_offline(pcap_file_path)
td_gen = parse_packets(packets_gen)
point_nums = []
for Td_map in tqdm(td_gen):
    point_nums.append((Td_map > 0.1).sum())
    Td_maps.append(Td_map)
bck_radius = 0.3
vertical_limits = [0,31]
lane_drawer = LaneDrawer() # lane drawer for queue detection
lane_drawer.update_lane_gdf()
lane_gdf = lane_drawer.lane_gdf
Td_maps = np.array(Td_maps)

# total_dis = []
# for i in tqdm(range(len(Td_maps))):
#     Td_map = Td_maps[i]
#     total_dis.append(Td_map.sum())

# total_dis = np.array(total_dis)
# mean_dis = total_dis.mean()
thred_map = gen_bckmap(Td_maps, N = 10,d_thred = 0.1,bck_n = 3)   
np.save('./thred_map.npy',thred_map)
thred_map = np.load('./thred_map.npy')

In [ ]:
Td_map_szie = Td_maps[0].shape
win_size = [7,13]
min_samples = 5
eps_dis = 0.5
db_fspc = Raster_DBSCAN(Td_map_szie=Td_map_szie, window_size=win_size,eps = eps_dis,min_samples=min_samples)
db = DBSCAN(eps=1, min_samples=5)
foregound_nums = []
time_fspc = []
time_dbscan = []
for i in tqdm(range(len(Td_maps))):
    Td_map = Td_maps[i]
    Foreground_map = ~(np.abs(Td_map - thred_map) <= bck_radius).any(axis = 0)
    point_cloud_data,labels = get_pcd_colored(Td_map,Foreground_map,vertical_limits)
    foreground_points = point_cloud_data[labels]
    time_a = time.time()
    Labels = db_fspc.fit_predict(Td_map,Foreground_map=Foreground_map)
    time_b = time.time()
    time_fspc.append(time_b - time_a)
    time_a = time.time()
    Labels = db.fit_predict(foreground_points)
    time_b = time.time()
    time_dbscan.append(time_b - time_a)
    foregound_nums.append((Foreground_map).sum())


In [ ]:
time_fspc = np.array(time_fspc) * 1000
time_dbscan = np.array(time_dbscan) * 1000
foregound_nums = np.array(foregound_nums)

In [ ]:
# Sample data (replace with actual data if available)
# np.random.seed(42)  # For reproducibility
# foregound_nums = np.linspace(1, 100, 100)  # Number of foreground points
# time_fspc = 0.05 * foregound_nums + np.random.normal(0, 0.1, len(foregound_nums))  # Linear relationship
# time_dbscan = 0.01 * foregound_nums * np.log(foregound_nums) + np.random.normal(0, 0.1, len(foregound_nums))  # Logarithmic relationship

# Define the curve fitting functions
def fspc_model(n, c, b):
    return c * n + b

def dbscan_model(n, c, b):
    return c * n * np.log(n) + b

# Fit the curves
popt_fspc, _ = curve_fit(fspc_model, foregound_nums, time_fspc)
popt_dbscan, _ = curve_fit(dbscan_model, foregound_nums, time_dbscan)

# Calculate fitted values
time_fspc_fit = fspc_model(foregound_nums, *popt_fspc)
time_dbscan_fit = dbscan_model(foregound_nums, *popt_dbscan)

# Calculate R-squared values
r2_fspc = r2_score(time_fspc, time_fspc_fit)
r2_dbscan = r2_score(time_dbscan, time_dbscan_fit)

# Plot the scatter and fitted curves
plt.figure(figsize=(5, 5))
plt.grid()
plt.scatter(foregound_nums, time_fspc, label='FSPC (Data)', s=3, alpha=0.3, c='r')
plt.scatter(foregound_nums, time_dbscan, label='DBSCAN (Data)', s=3, alpha=0.3, c='b')
# plt.plot(foregound_nums, time_fspc_fit, label=f'FSPC Fit (R²={r2_fspc:.2f})', c='darkred')
# plt.plot(foregound_nums, time_dbscan_fit, label=f'DBSCAN Fit (R²={r2_dbscan:.2f})', c='darkblue')
plt.legend()
plt.xlabel('Number of Foreground Points (n)')
plt.ylabel('Time')
# plt.title('Curve Fitting and R² Calculation')
plt.show()


In [ ]:
from shapely.geometry import box
# Define the grid parameters
x_min, x_max = -50, 50  # Range for x-coordinates
y_min, y_max = -50, 50  # Range for y-coordinates
grid_size = 0.5  # Grid size (0.5m x 0.5m)

# Generate grid boxes
grid_boxes = [
    box(
        x_min + x * grid_size,
        y_min + y * grid_size,
        x_min + (x + 1) * grid_size,
        y_min + (y + 1) * grid_size,
    )
    for x in range(int((x_max - x_min) / grid_size))
    for y in range(int((y_max - y_min) / grid_size))
]

# Create a GeoDataFrame using the grid boxes
grid_gdf = gpd.GeoDataFrame({'geometry': grid_boxes})

In [ ]:
plt.figure(figsize=(5, 5))
plt.grid()
plt.scatter(point_nums, time_rtree, label='Traditional BF', s=3, alpha=0.3, c='r')
plt.scatter(point_nums, time_bf, label='Proposed BF', s=3, alpha=0.3, c='b')
plt.legend()
plt.xlabel('Number of Points')
plt.ylabel('Time (ms)')
plt.ylim(0, 200)
plt.show()